In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.python.saved_model import tag_constants

from setlearn.data import modelnet as dmodel

print( "The tensorflow version is {}.".format( tf.__version__ ) )

The tensorflow version is 1.11.0.


In [2]:
EPOCHS = 50

In [3]:
model_path = './model_test_43/'
data_path  = './data/ModelNet38_cloud.h5'

batch_size = 128

In [4]:
graph_ld = tf.Graph()
with graph_ld.as_default():
    with tf.Session( graph=graph_ld ) as sess:
        print( "Restoring model..." )
        tf.saved_model.loader.load(
            sess,
            [tag_constants.SERVING],
            model_path
        )
        print( 'restored!')
        
        inputs_ph      = graph_ld.get_tensor_by_name( 'inputs:0' )
        cards_ph       = graph_ld.get_tensor_by_name( 'cards:0' )
        ys_ph          = graph_ld.get_tensor_by_name( 'ys:0' )
        is_training_ph = graph_ld.get_tensor_by_name( 'is_training:0' )
        
        features = graph_ld.get_tensor_by_name( 'permequiv3/point_feats/fc_net/out/bias:0' )
        
        tmp0 = graph_ld.get_tensor_by_name( 'SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits:0' )
        tmp1 = graph_ld.get_tensor_by_name( 'fc_net/add:0')
        
#         print( graph_ld.as_graph_def() )
#         print( tf.all_variables() )
#         print( tf.trainable_variables() )

Restoring model...
INFO:tensorflow:Restoring parameters from ./model_test_43/variables/variables
restored!


In [5]:
features

<tf.Tensor 'permequiv3/point_feats/fc_net/out/bias:0' shape=(256,) dtype=float32_ref>

In [6]:
rfetcher = dmodel.ModelFetcher( 
    data_path, 
    batch_size=batch_size, 
)
fixed_features = tf.stop_gradient( features )

graph_ld = tf.Graph()
with graph_ld.as_default():
    with tf.Session( graph=graph_ld ) as sess:
        print( "Restoring model..." )
        tf.saved_model.loader.load(
            sess,
            [tag_constants.SERVING],
            model_path
        )
        print( 'restored!')
        
        inputs_ph      = graph_ld.get_tensor_by_name( 'inputs:0' )
        cards_ph       = graph_ld.get_tensor_by_name( 'cards:0' )
        ys_ph          = graph_ld.get_tensor_by_name( 'ys:0' )
        is_training_ph = graph_ld.get_tensor_by_name( 'is_training:0' )
        
        features = graph_ld.get_tensor_by_name( 'permequiv3/point_feats/fc_net/out/bias:0' )
        
        for ee in range( EPOCHS ):
            

In [7]:
loss_val = 0 # initialize loss value
with graph_ld.as_default():
    with tf.Session(graph=graph_ld) as sess:
        
        print( "Restoring model..." )
        tf.saved_model.loader.load(
            sess,
            [tag_constants.SERVING],
            model_path
        )
        print( 'restored!')
        
        inputs_ph      = graph_ld.get_tensor_by_name( 'inputs:0' )
        cards_ph       = graph_ld.get_tensor_by_name( 'cards:0' )
        ys_ph          = graph_ld.get_tensor_by_name( 'ys:0' )
        is_training_ph = graph_ld.get_tensor_by_name( 'is_training:0' )
        
        features = tf.stop_gradient(
            graph_ld.get_tensor_by_name( 'tens/reduce/normed_tensor:0' )
        )
        
        labels = tf.one_hot( ys_ph, depth=40 )
        logits = graph_ld.get_tensor_by_name( 'fc_net/add:0' )
        
        loss = tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=labels
        )
        
        avg_loss = tf.reduce_mean( loss )
        
        for ee in range( 1 ): # EPOCHS ):
            
            train_data = rfetcher.train_data( loss_val )
            for inps, cs, ms in train_data:
                feed_dict = {
                    inputs_ph: inps, 
                    cards_ph: cs, 
                    ys_ph: ms, 
                    is_training_ph: False
                }
                logits_, labels_, loss_, feat_ = sess.run( 
                    [logits, labels, avg_loss, features], 
                    feed_dict=feed_dict
                )
                
print( 'Finished training!' )

Restoring model...
INFO:tensorflow:Restoring parameters from ./model_test_43/variables/variables
restored!
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Train loss: 0.0000: 100%|██████████| 67/67 [00:03<00:00, 19.13it/s]

Finished training!


In [8]:
loss_val

0

In [9]:
np.shape( loss_val )

()